# Analysis of the RWC 2027 draw

This is the code used for this substack post: https://dimitriperrin.substack.com/p/so-what-do-we-make-of-that-rugby

## Background data

We start by loading the rankings from a file. This is better than hard-coding them, in case we want to repeat the analysis closer to the start of the World Cup (with updated ranking points).

In [1]:
import csv

rankings = {}

with open("WR_rankings_Dec2025.csv", newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        key, value = row
        rankings[key] = float(value)

print(rankings)


{'South Africa': 93.94, 'New Zealand': 90.33, 'England': 89.41, 'Ireland': 87.97, 'France': 87.24, 'Argentina': 84.97, 'Australia': 81.53, 'Fiji': 81.14, 'Scotland': 80.22, 'Italy': 78.98, 'Wales': 74.23, 'Japan': 74.09, 'Georgia': 73.18, 'Uruguay': 69.19, 'Spain': 69.01, 'USA': 68.26, 'Chile': 66.72, 'Tonga': 66.66, 'Samoa': 66.43, 'Portugal': 64.98, 'Romania': 61.5, 'Hong Kong China': 59.61, 'Zimbabwe': 58.8, 'Canada': 58.75}


On the other hand, the bands for the draw are fixed, so we can directly code them in.

In [2]:
bands = {
    "Band 1": ['South Africa', 'New Zealand', 'England', 'Ireland', 'France', 'Argentina'],
    "Band 2": ['Australia', 'Fiji', 'Scotland', 'Italy', 'Wales', 'Japan'],
    "Band 3": ['Georgia', 'Uruguay', 'Spain', 'USA', 'Chile', 'Tonga'],
    "Band 4": ['Samoa', 'Portugal', 'Romania', 'Hong Kong China', 'Zimbabwe', 'Canada']
}

## RWC 2027 pools

Now that the draw is made, pools are fixed as well, so we can code them in as well.

In [3]:
pools = {
    "A": ['New Zealand', 'Australia', 'Chile', 'Hong Kong China'],
    "B": ['South Africa', 'Italy', 'Georgia', 'Romania'],
    "C": ['Argentina', 'Fiji', 'Spain', 'Canada'],
    "D": ['Ireland', 'Scotland', 'Uruguay', 'Portugal'],
    "E": ['France', 'Japan', 'USA', 'Samoa'],
    "F": ['England', 'Wales', 'Tonga', 'Zimbabwe']
}

team_to_pool = {
    'South Africa': "B",
    'New Zealand': "A",
    'England': "F", 
    'Ireland': "D", 
    'France': "E", 
    'Argentina': "C",
    
    'Australia': "A",
    'Fiji': "C", 
    'Scotland': "D", 
    'Italy': "B", 
    'Wales': "F", 
    'Japan': "E", 
    
    'Georgia': "B",
    'Uruguay': "D", 
    'Spain': "C", 
    'USA': "E", 
    'Chile': "A", 
    'Tonga': "F",
    
    'Samoa': "E",
    'Portugal': "D",
    'Romania': "B",
    'Hong Kong China': "A",
    'Zimbabwe': "F",
    'Canada': "C"
}

## Pool difficulty

### Total points per pool

In [4]:
pool_scores = {
    pool_name: sum(rankings.get(team, 0) for team in teams)
    for pool_name, teams in pools.items()
}

sorted_pools = sorted(pool_scores.items(), key=lambda x: x[1], reverse=True)

for pool, score in sorted_pools:
    print(f"{pool}: {score:.2f}")


B: 307.60
D: 302.36
A: 298.19
E: 296.02
C: 293.87
F: 289.10


### Relative difficulty, per band

In [5]:
results_by_band = {}

for band_name, teams in bands.items():
    team_values = {}

    for team in teams:
        # First, we identify pool
        pool = team_to_pool[team]
        pool_teams = pools[pool]

        # Then we compute the sum of raw pairwise differences
        team_points = rankings[team]

        value = sum(
            team_points - rankings[other]
            for other in pool_teams
            if other != team
        )

        team_values[team] = value

    # Finally, we sort all teams within the band
    sorted_teams = sorted(team_values.items(), key=lambda x: x[1], reverse=True)

    results_by_band[band_name] = sorted_teams

# Output
for band_name, sorted_teams in results_by_band.items():
    print(f"\n{band_name}:")
    for team, value in sorted_teams:
        print(f"  {team}: {value:.2f}")



Band 1:
  England: 68.54
  South Africa: 68.16
  New Zealand: 63.13
  France: 52.94
  Ireland: 49.52
  Argentina: 46.01

Band 2:
  Fiji: 30.69
  Australia: 27.93
  Scotland: 18.52
  Italy: 8.32
  Wales: 7.82
  Japan: 0.34

Band 3:
  Georgia: -14.88
  Spain: -17.83
  Tonga: -22.46
  USA: -22.98
  Uruguay: -25.60
  Chile: -31.31

Band 4:
  Samoa: -30.30
  Portugal: -42.44
  Zimbabwe: -53.90
  Canada: -58.87
  Hong Kong China: -59.75
  Romania: -61.60


## Pool outcomes

In [6]:
pool_results = {}

for pool_name, pool_teams in pools.items():
    team_values = {}

    for team in pool_teams:
        team_points = rankings[team]

        value = sum(
            team_points - rankings[other]
            for other in pool_teams
            if other != team
        )

        team_values[team] = value

    # Sort in decreasing order
    sorted_teams = sorted(team_values.items(), key=lambda x: x[1], reverse=True)

    pool_results[pool_name] = sorted_teams


# Display
for pool_name, sorted_teams in pool_results.items():
    print(f"\n{pool_name}:")
    for team, value in sorted_teams:
        print(f"  {team}: {value:.2f}")



A:
  New Zealand: 63.13
  Australia: 27.93
  Chile: -31.31
  Hong Kong China: -59.75

B:
  South Africa: 68.16
  Italy: 8.32
  Georgia: -14.88
  Romania: -61.60

C:
  Argentina: 46.01
  Fiji: 30.69
  Spain: -17.83
  Canada: -58.87

D:
  Ireland: 49.52
  Scotland: 18.52
  Uruguay: -25.60
  Portugal: -42.44

E:
  France: 52.94
  Japan: 0.34
  USA: -22.98
  Samoa: -30.30

F:
  England: 68.54
  Wales: 7.82
  Tonga: -22.46
  Zimbabwe: -53.90


In [7]:
placements = {}

third_place_candidates = []   # list of (pool_name, team, points)

for pool_name, pool_teams in pools.items():
    # Sort teams by decreasing points
    sorted_teams = sorted(pool_teams, key=lambda t: rankings[t], reverse=True)

    # First-ranked team
    placements[f"{pool_name}1"] = sorted_teams[0]

    # Second-ranked team
    placements[f"{pool_name}2"] = sorted_teams[1]

    # Third-ranked team (saved for later global selection)
    if len(sorted_teams) >= 3:
        third_place_candidates.append(
            (pool_name, sorted_teams[2], rankings[sorted_teams[2]])
        )

# Now select the top 4 third-place teams globally
third_place_candidates.sort(key=lambda x: x[2], reverse=True)
top_four_thirds = third_place_candidates[:4]

# Store them under keys "3rd1", "3rd2", "3rd3", "3rd4"
for i, (pool, team, points) in enumerate(top_four_thirds, start=1):
    placements[f"3rd{i}"] = {"pool": pool, "team": team}

placements

{'A1': 'New Zealand',
 'A2': 'Australia',
 'B1': 'South Africa',
 'B2': 'Italy',
 'C1': 'Argentina',
 'C2': 'Fiji',
 'D1': 'Ireland',
 'D2': 'Scotland',
 'E1': 'France',
 'E2': 'Japan',
 'F1': 'England',
 'F2': 'Wales',
 '3rd1': {'pool': 'B', 'team': 'Georgia'},
 '3rd2': {'pool': 'D', 'team': 'Uruguay'},
 '3rd3': {'pool': 'C', 'team': 'Spain'},
 '3rd4': {'pool': 'E', 'team': 'USA'}}

## Round of 16

In [8]:
from itertools import permutations

# Constraints for each top-1 team
constraints = {
    "A1": {"C", "E", "F"},
    "B1": {"D", "E", "F"},
    "C1": {"A", "E", "F"},
    "D1": {"B", "E", "F"},
}

# Extract list of third-placed teams
third_teams = [
    (placements[key]["pool"], placements[key]["team"])
    for key in ["3rd1", "3rd2", "3rd3", "3rd4"]
]

solutions = []

# Try all permutations of third-placed teams
for perm in permutations(third_teams):
    valid = True
    pairing = {}
    for top_team, third in zip(["A1", "B1", "C1", "D1"], perm):
        pool, team = third
        if pool not in constraints[top_team]:
            valid = False
            break
        pairing[top_team] = team
    if valid:
        solutions.append(pairing)

# Check the number of solutions - just for safety (it will always be unique)
if len(solutions) == 0:
    print("Error: No valid solution exists for the round-of-16 fixtures.")
elif len(solutions) > 1:
    print("Error: Multiple valid solutions exist for the round-of-16 fixtures.")
    
# 'solutions[0]' contains the unique mapping for top-1 teams to third-placed teams
top_vs_third = solutions[0]

# Other pre-determined matches, using placements to get actual team names
other_matches = [
    (placements["C2"], placements["F2"]),
    (placements["E1"], placements["D2"]),
    (placements["A2"], placements["E2"]),
    (placements["F1"], placements["B2"])
]

# Build fixtures dictionary
fixtures = {}

# Round-of-16 order
fixtures["R1"] = (placements["A1"], top_vs_third["A1"])
fixtures["R2"] = (placements["B1"], top_vs_third["B1"])
fixtures["R3"] = other_matches[0]  # C2 vs F2
fixtures["R4"] = other_matches[1]  # E1 vs D2
fixtures["R5"] = other_matches[2]  # A2 vs E2
fixtures["R6"] = other_matches[3]  # F1 vs B2
fixtures["R7"] = (placements["C1"], top_vs_third["C1"])
fixtures["R8"] = (placements["D1"], top_vs_third["D1"])

# Print nicely
for round_name, match in fixtures.items():
    print(f"{round_name}: {match[0]} vs {match[1]}")


R1: New Zealand vs Spain
R2: South Africa vs Uruguay
R3: Fiji vs Wales
R4: France vs Scotland
R5: Australia vs Japan
R6: England vs Italy
R7: Argentina vs USA
R8: Ireland vs Georgia


## Quarter finals

In [9]:
def get_winner(team1, team2):
    """Return the team with the higher ranking points."""
    return team1 if rankings[team1] >= rankings[team2] else team2

# Define the QF matchups in terms of R1-R8
qf_matchups = [
    ("R1", "R2"),  # QF1
    ("R3", "R4"),  # QF2
    ("R5", "R6"),  # QF3
    ("R7", "R8")   # QF4
]

# Generate the quarter-final fixtures
qf_fixtures = {}
for i, (r1, r2) in enumerate(qf_matchups, start=1):
    winner1 = get_winner(*fixtures[r1])
    winner2 = get_winner(*fixtures[r2])
    qf_fixtures[f"QF{i}"] = (winner1, winner2)

# Print nicely
for qf, match in qf_fixtures.items():
    print(f"{qf}: {match[0]} vs {match[1]}")


QF1: New Zealand vs South Africa
QF2: Fiji vs France
QF3: Australia vs England
QF4: Argentina vs Ireland


## Semi finals

In [10]:
# Define SF matchups in terms of QFs
sf_matchups = [
    ("QF1", "QF2"),  # SF1
    ("QF3", "QF4")   # SF2
]

# Generate semi-final fixtures
sf_fixtures = {}
for i, (qf1, qf2) in enumerate(sf_matchups, start=1):
    winner1 = get_winner(*qf_fixtures[qf1])
    winner2 = get_winner(*qf_fixtures[qf2])
    sf_fixtures[f"SF{i}"] = (winner1, winner2)

# Print nicely
for sf, match in sf_fixtures.items():
    print(f"{sf}: {match[0]} vs {match[1]}")


SF1: South Africa vs France
SF2: England vs Ireland


## Final

In [11]:
def get_loser(team1, team2):
    """Return the team with the lower ranking points."""
    return team1 if rankings[team1] < rankings[team2] else team2

# Semi-final winners and losers
sf1_winner = get_winner(*sf_fixtures["SF1"])
sf1_loser  = get_loser(*sf_fixtures["SF1"])
sf2_winner = get_winner(*sf_fixtures["SF2"])
sf2_loser  = get_loser(*sf_fixtures["SF2"])

# Third-place match
third_place_match = (sf1_loser, sf2_loser)
third_place_winner = get_winner(*third_place_match)

# Final
final_match = (sf1_winner, sf2_winner)
final_winner = get_winner(*final_match)

# Print results
print(f"Third-place match: {third_place_match[0]} vs {third_place_match[1]}")
print(f"Winner: {third_place_winner}\n")

print(f"Final: {final_match[0]} vs {final_match[1]}")
print(f"Winner: {final_winner}")


Third-place match: France vs Ireland
Winner: Ireland

Final: South Africa vs England
Winner: South Africa


Done.